In [ ]:
import pandas as pd
import numpy as np
import pyarrow.dataset as ds

from utils import calc_expected_score, calc_new_rating

In [ ]:
data_path = '../../Users/justynrodrigues/Documents/nfl/pbp/'
dataset = ds.dataset(data_path, format='parquet')
# game_id_df = dataset.to_table(columns = ['game_id', 'home_team', 'away_team', 'season_type', 'week', 'game_date']).to_pandas().drop_duplicates(subset = ['game_id']).reset_index(drop = True)


In [ ]:
#nfl_games_19 = pd.read_csv("http://www.stat.cmu.edu/cmsac/football/data/nfl_games_2019.csv") 

nfl_games_19 = pd.read_csv('~/Downloads/nfl_games_2019.csv')

nfl_games_19 = nfl_games_19.assign(
    game_outcome = lambda x: np.where(x['home_score'] > x['away_score'], 1, np.where(x['home_score'] == x['away_score'], 0.5, 0))
)

# nfl_games_19["game_outcome"] = np.where(nfl_games_19["home_score"] > nfl_games_19["away_score"], 1, np.where(nfl_games_19["home_score"] == nfl_games_19["away_score"], 0.5, 0))

nfl_elo_ratings = pd.read_csv('~/Downloads/nfl_538_init_elo_ratings.csv')

#nfl_elo_ratings = pd.read_csv("http://www.stat.cmu.edu/cmsac/football/data/nfl_538_init_elo_ratings.csv")   

In [ ]:
df = pd.read_csv('//Users/justynrodrigues/Documents/nfl/data/game_results.csv')

df = (
    df[(df.season < 2022)]
    [['game_id', 'gameday', 'game_type', 'season', 'home_team', 'away_team', 'home_score', 'away_score', 'result', 'home_moneyline', 'away_moneyline', 'spread_line', 'away_spread_odds', 'home_spread_odds']]
    .assign(implied_home_win = lambda x: money_line_odds(x['home_moneyline'], x['away_moneyline'])[0])
    .dropna()
    .reset_index(drop = True)
    .merge(game_id_df, how = 'left', left_on = ['game_id', 'home_team', 'away_team'], right_on = ['game_id', 'home_team', 'away_team'])
    .assign(game_date = lambda x: pd.to_datetime(x['game_date']))
    .drop(['game_type', 'gameday'], axis = 1)
    )

# game_id, home_team, away_team, week, season, home_score, away_score || nfl_games_19

In [ ]:
for game_i in range(nfl_games_19.shape[0]):
    # Which teams are we looking at?
    home_team = nfl_games_19.iloc[game_i]["home_team"]
    away_team = nfl_games_19.iloc[game_i]["away_team"]
    # What was the observed score by the home team?
    home_score = nfl_games_19.iloc[game_i]["game_outcome"]
    # Week number?
    game_week = nfl_games_19.iloc[game_i]["week"]

    # What was each team's rating from their latest game?
    home_rating = nfl_elo_ratings.loc[nfl_elo_ratings["team"] == home_team].sort_values(by = "week", ascending = False).iloc[0]["elo_rating"]
    away_rating = nfl_elo_ratings.loc[nfl_elo_ratings["team"] == away_team].sort_values(by = "week", ascending = False).iloc[0]["elo_rating"]

    # Now get their new ratings:
    new_home_rating = calc_new_rating(home_rating, home_score, calc_expected_score(home_rating, away_rating))
    # Opposite for away team:
    new_away_rating = calc_new_rating(away_rating, 1 - home_score, calc_expected_score(away_rating, home_rating))

    # Finally - join to the nfl_elo_ratings table each team's new ratings for the week:
    updated_ratings = pd.DataFrame(data = {
        "team": [home_team, away_team],
        "elo_rating": [new_home_rating, new_away_rating],
        "week": [game_week, game_week]})
    nfl_elo_ratings = nfl_elo_ratings.append(updated_ratings)